Ref:
1. RegEx
    https://www.tutorialspoint.com/python/python_reg_expressions.htm


In [2]:
import os
import re
import pandas as pd

In [2]:
os.listdir(path='./CloudImg')

['.ipynb_checkpoints',
 'satellite-water-vapor-10min',
 'Top alert (every 10min)',
 'CropCloud',
 'HD']

In [3]:
def detect_per_month(lst_path ,days=31):

    per_month = []
    days = days
    for u in range(1, days+1):
        per_month.append([i for i in lst_path if int(i[0][-2:])==u])

    lst_lack = []
    len_lack = []
    rep = []

    for u in range(days):
        if len(per_month[u]) != 24:
            len_lack.append([per_month[u][0][0], 24-len(per_month[u])])

        for i in range(len(per_month[u])):
            now = per_month[u][i][-1]

            if (i == 0) & (int(now) != 0):
                for v in range(0, int(now)):
                    lst_lack.append(per_month[u][i][0] + '_' + str(v))

            if i >= 1:
                before = per_month[u][i-1][-1]
                if (int(now) - int(before)) == 0:
                    rep.append(per_month[u][i][0] + '_' + now)
                if (int(now) - int(before)) > 1:
                    for v in range(1, (int(now) - int(before))):
                        lst_lack.append(per_month[u][i][0] + '_' + str(int(before)+v))

            if (i+1 == len(per_month[u])) & (int(now) != 23):
                for v in range(0, (23 - int(now))):
                    lst_lack.append(per_month[u][i][0] + '_' + str(int(now)+v+1))

    print(f'lack of hours:\n {lst_lack}', end='\n\n')
    print(f'repeated hours:\n {rep}', end='\n\n')
    #print(f'short length day and the lack:\n {len_lack}', end='\n\n')
    #print(f'lack of lenth: {24*days - len(lst_path)}', end='\n\n')
    print('='*80)

In [4]:
def detect_month(img='satellite-water-vapor-10min', months=[1]):
    
    print(f'{img}', end='\n\n')
    months=months
    lst_per_month=[]

    lst_path = os.listdir(path='./'+img)
    lst_path = sorted([re.search(r'\d{4}_\d{2}_\d{2}_\d{2}', i).group() for i in lst_path if i != '.ipynb_checkpoints'])
    lst_path = [i.split('_') for i in lst_path]
    lst_path = [['_'.join(i[0:3]), i[-1]] for i in lst_path]
    #print(lst_path)

    for month in months:
        lst_per_month.append([i for i in lst_path if int(i[0][5:7])==month])
    #print(lst_per_month)
    
    for v in range(len(months)):
        detect_per_month(lst_per_month[v],days=int(lst_per_month[v][-1][0][-2:]))

In [5]:
for i in ['satellite-hd-10min']:
    detect_month(i, months=[1])#1,2,3,4,5,6,7,8,9,10,11,12

satellite-hd-10min

lack of hours:
 []

repeated hours:
 []



In [6]:
# after revision, check the integrity...
def integrity(img='HD'):
    lst_path = os.listdir(path='./CloudImg/'+img)
    lst_path = sorted([re.search(r'\d{4}_\d{2}_\d{2}_\d{2}', i).group() for i in lst_path if i != '.ipynb_checkpoints'])
    lst_path = [i.split('_') for i in lst_path]
    lst_path = pd.DataFrame(lst_path)
    #print(lst_path)
    
    days = lst_path[2].nunique()
    #print(days)

    print(sum(lst_path.groupby([2]).count()[3]==24) == days)

In [7]:
integrity()

True


In [ ]:
'''
satellite-water-vapor-10min

lack of hours:
 ['2017_01_05_19', '2017_01_06_7', '2017_01_11_6', '2017_01_22_21', '2017_01_22_23']

repeated hours:
 ['2017_01_05_18', '2017_01_06_06', '2017_01_11_05', '2017_01_22_20']

================================================================================
Top alert (every 10min)

lack of hours:
 ['2017_01_05_19', '2017_01_06_7', '2017_01_11_4', '2017_01_11_5', '2017_01_11_7', '2017_01_15_6', '2017_01_20_20', '2017_01_22_21', '2017_01_22_23', '2017_01_23_0', '2017_01_27_19', '2017_01_29_12', '2017_01_30_12']

repeated hours:
 ['2017_01_05_18', '2017_01_06_06', '2017_01_11_03', '2017_01_11_06', '2017_01_11_06', '2017_01_22_20']

================================================================================
HD (every 10min)

lack of hours:
 ['2017_01_01_0', '2017_01_05_19', '2017_01_06_7', '2017_01_10_21', '2017_01_11_4', '2017_01_11_7', '2017_01_16_8', '2017_01_18_6', '2017_01_22_21', '2017_01_23_18', '2017_01_25_13', '2017_01_29_7', '2017_01_29_16']

repeated hours:
 ['2017_01_05_18', '2017_01_06_06', '2017_01_11_03', '2017_01_11_05', '2017_01_22_20']

================================================================================
'''